## resnet 모델을 통한 아기울음 소리(.wav) 원인 분류


In [1]:
# Load packages
import numpy as np
import tensorflow as tf

### 아기울음 소리 분류에 사용될 모델을 불러온다.


In [2]:
model_path = '/Users/jaewone/developer/tensorflow/baby-cry-classification/model/resnet.h5'
model = tf.keras.models.load_model(model_path)
model.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-08-21 22:33:30.529912: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-21 22:33:30.530251: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 862, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 868, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 431, 64)  9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 431, 64)  256         ['conv1_conv[0][0]']       

### wav 파일을 통해 모델의 input vector를 생성한다.


In [3]:
import librosa
from skimage.transform import resize

classes = ['sad', 'hug', 'diaper', 'hungry',
           'sleepy', 'awake', 'uncomfortable']


def get_input_vector_from_file(file_path: str) -> np.ndarray:
    y, sr = librosa.load(file_path, sr=16000)
    mel_spec = librosa.feature.melspectrogram(
        y=y, sr=sr, n_mels=128, n_fft=2048, hop_length=501)
    mel_spec_dB = librosa.power_to_db(mel_spec, ref=np.max)
    RATIO = 862 / 64
    mel_spec_dB_resized = resize(mel_spec_dB, (mel_spec_dB.shape[0], mel_spec_dB.shape[1] * RATIO),
                                 anti_aliasing=True, mode='reflect')
    mel_spec_dB_stacked = np.stack([mel_spec_dB_resized] * 3, axis=-1)
    return mel_spec_dB_stacked[np.newaxis, ]


test_vector = get_input_vector_from_file(
    '/Users/jaewone/developer/tensorflow/baby-cry-classification/data/diaper/diaper_18.wav')
test_vector.shape

/Users/jaewone/ENTER/envs/tf25/lib/python3.9/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


(1, 128, 862, 3)

### 예측을 수행한다.


In [4]:
predictions = model.predict(test_vector)[0]
classes[np.argmax(predictions)]

2023-08-21 22:33:34.410976: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-21 22:33:34.677729: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


'diaper'